In [67]:
from scipy.spatial.distance import euclidean 
from IPython.core.debugger import set_trace
import numpy as np 
from sklearn.preprocessing import scale

In [2]:
A = np.array([0,0,0,1,-1,2,2,0,3,0,4,-1]).reshape(-1,2) 

In [68]:
def sq_eucl_dist(x,y,sparse=True):
        """
        Perform the square euclidien Distance 
        """
        if sparse:
            ind = np.intersect1d(x.nonzero(),y.nonzero())
            sq = pow((x[ind]-y[ind]),2)
            return np.sum(sq)
        else:
            sq = pow((x-y),2)
            return np.sum(sq)


def cent_init(A , K):
    cent_list = defaultdict()
    index = np.arange(A.shape[0])
    cent1 = np.random.choice(index)
    cent_list[cent1]= A[cent1]
    
    while (K-len(cent_list))>0:
        max_indece = None
        max_dis = 0
        for ind, el in enumerate(A):
            dist = 0
            if ind not in cent_list:
                for cent in cent_list:
                        dist += sq_eucl_dist(A[cent],el)
            if dist > max_dis:
                max_indece = ind
                max_dis = dist
        cent_list[max_indece]=A[max_indece]
                    
    return cent_list
                
from collections import defaultdict

def k_mean(A,init):
    kluster_list = list(init.keys())
    cluster_element = defaultdict(list)
    J_clust = 0
    for num, el in enumerate(A):
        list_dist = []
        for kluster in kluster_list:
            list_dist.append(sq_eucl_dist(el,init[kluster]))
        close_clt = np.argmin(list_dist)
        J_clust += min(list_dist)
        cluster_element[kluster_list[close_clt]].append(num)
    return cluster_element, J_clust/(A.shape[0])


def mean_(x):
    return np.mean(x,axis=0)

def Weizfeld_median(A, max_iter):
    "Return the geometric median "
    mu_0 = mean_(A)
    for it in range(max_iter):
        t = np.array(list(sum(x) for x in zip(*np.array([x*np.sqrt(sq_eucl_dist(x,mu_0))**(-1) for x in A]))))
        mu =t/ sum(np.sqrt(sq_eucl_dist(x,mu_0))**(-1) for x in A)
        if abs(sum(mu_0-mu)) < 10**(-10):
            return mu
        mu_0 = mu
    return mu

def comput_centroid(A, cluster_list, geom_median=True,max_iter=1000):
    """
    This compute the entroide of the clusters
    if geom_median is set to True: it computs the geometric median 
    otherwise the mean is calculted 
    """
    centroid_value = defaultdict()
    if geom_median:
        for centroid, element in cluster_list.items():
            if len(element)>1:
                mu = Weizfeld_median(A[element].squeeze(), max_iter)
                centroid_value[centroid]= mu
            else:
                centroid_value[centroid]= A[centroid]
        return centroid_value
    
    for centroid, element in cluster_list.items():
        if len(element)>1:
            mu = mean_(A[element].squeeze())
            centroid_value[centroid]= mu
        else:
            centroid_value[centroid]= A[centroid]
    return centroid_value


def train(A,K, thresold=10**(-60)):
    initial = cent_init(A,K)
    converg = True
    distortion = []
    it = 0
    while converg:
        centroid, J_clt = k_mean(A, initial)
        new = comput_centroid(A, centroid)
        distortion.append(J_clt)
        sum_cent = 0
        for cent in list(new.keys()):
            a = np.array(initial[cent])
            b= np.array(new[cent])
            sum_cent += sum(abs(a-b))
        if sum_cent< thresold:
            converg=False
            print('initial',new)
            print("converge")
        initial = new
        it +=1
    return initial, centroid
        
        
    
    



In [69]:
sq_eucl_dist(A[2],A[3])

9

In [70]:
def normilize(x):
    mean = np.mean(x, axis=0)
    std = np.std(x,axis=0)
    return (x-mean)/std

In [72]:
A_sc = normilize(A)

In [73]:
A_sk = scale(A, axis=0, with_mean=True, with_std=True, copy=True)

/Users/souayboubagayoko/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [74]:
A_sc

array([[-0.74278135, -0.35355339],
       [-0.74278135,  0.70710678],
       [-1.29986737,  1.76776695],
       [ 0.37139068, -0.35355339],
       [ 0.92847669, -0.35355339],
       [ 1.48556271, -1.41421356]])

In [75]:
A_sk

array([[-0.74278135, -0.35355339],
       [-0.74278135,  0.70710678],
       [-1.29986737,  1.76776695],
       [ 0.37139068, -0.35355339],
       [ 0.92847669, -0.35355339],
       [ 1.48556271, -1.41421356]])

In [59]:
lout2

[array([0]), array([0]), array([-1]), array([2]), array([3]), array([4])]

In [60]:
lout1

[array([0]), array([0]), array([-1]), array([2]), array([3]), array([4])]

In [62]:
A.T

array([[ 0,  0, -1,  2,  3,  4],
       [ 0,  1,  2,  0,  0, -1]])